In [1]:
import pandas as pd
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
df = pd.read_csv(outputFilePath+'user_category'+".csv")
print('end')

end


medium qn 2 : find the compliment count for a user review

script to insert table to db

In [8]:
import pandas as pd

def extractDfFromFile(nFiles, filePrefix, headerFlag):
    df = pd.DataFrame()
    for i in range(1,nFiles+1): # loops from 1 to nFiles
        if headerFlag == 0:
            entries = pd.read_csv(filePrefix+str(i)+'.txt', sep = '@', header = None)
        else:
            entries = pd.read_csv(filePrefix+str(i)+'.txt', sep = '@')
        df = df.append(pd.DataFrame(entries))
    return df

In [22]:
filePath = '/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/'
filePrefix = filePath+'tips_'
tipsDf = extractDfFromFile(13, filePrefix, 1)

#aggregate compliment count
groupedTipsDf = tipsDf.groupby(['user_id']).sum().reset_index()
print('end')

end


In [ ]:
from cassandra.cluster import Cluster
import pandas as pd

if __name__ == "__main__":
    cluster = Cluster(['127.0.0.1'], port = 9042)
    session = cluster.connect('yelp_weather_data',wait_for_all_pools=True) #json - name of keyspace
    session.execute('USE yelp_weather_data')

    query = "INSERT INTO user_tip_comp_count(userid , complimentcount) VALUES (?,?)"
    prepared = session.prepare(query)

    for index, item in groupedTipsDf.iterrows():
        print(index)
        #print(item)
        session.execute(prepared, (item['user_id'],int(item['compliment_count'])))
    
#     rows = session.execute('SELECT * FROM checkin')
#     for row in rows:
#         #print(row)
#         print("INSERT INTO checkin(businessid,dates) VALUES (","'"+row.businessid+"'", ",'"+row.dates+"'",");")



Script to insert User_recommendation table in db

Steps:

1. For a user find max rated category

2. Find max correlated category of this category

3. Return the user with highest rating in the above category

In [ ]:
import pandas as pd
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
df = pd.read_csv(outputFilePath+'user_category'+".csv")
print('end')
vals = df.drop(['user_id'], axis = 1)

In [24]:
cat_corrs = df.corr(method='pearson')
print('end')

In [34]:
max_rating_category_all = vals.idxmax(axis=1)
df['max_user_category'] = max_rating_category_all 

In [93]:
import numpy as np
np.fill_diagonal(cat_corrs.values, 0)

# because we dont want the max correlated category of a given category to be miscellaneous
cat_corrs_no_mis = cat_corrs.drop(index=['Miscelleanous']) #drops last row
print(cat_corrs_no_mis.shape)
# cat_corrs_no_mis = cat_corrs_no_mis.drop(columns=['Miscelleanous'])
# print(cat_corrs_no_mis.shape) 

max_corr_categories_series = cat_corrs_no_mis.idxmax()
df['max_corr_category'] = df['max_user_category'].map(max_corr_categories_series)

#print(df['max_corr_category'])

(32, 33)
0            Transportation
1          Health & Medical
2                      Bars
3                      Bars
4          Health & Medical
                 ...       
1637133                Bars
1637134                Bars
1637135                Bars
1637136                Bars
1637137                Bars
Name: max_corr_category, Length: 1637138, dtype: object


In [ ]:
user_id_series = df['user_id']  

category_max_user_index = vals.idxmax(axis = 0)  # keys : 0 to 1637138..
category_2nd_max_user_index = vals[ vals.apply(lambda x: (x!=max(x)))].idxmax(axis = 0)
#print(category_max_user_index)

category_max_userid_map = category_max_user_index.map(user_id_series)  #keys actual user ids
category_2nd_max_userid_map = category_2nd_max_user_index.map(user_id_series)

df['max_corr_user'] = df['max_corr_category'].map(category_max_userid_map)
second_max_corr_user = df['max_corr_category'].map(category_2nd_max_userid_map)
df['second_max_corr_user'] = second_max_corr_user

df['max_corr_user'] = df.apply(lambda x: x['second_max_corr_user'] 
                               if x['max_corr_user'] == x['user_id'] else x['max_corr_user'], axis =1)
#print(df.head())

In [115]:
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
df.to_csv(outputFilePath+'user_category_user.csv', index=False)
print('end')

end


In [ ]:
from cassandra.cluster import Cluster
import pandas as pd

if __name__ == "__main__":
    cluster = Cluster(['127.0.0.1'], port = 9042)
    session = cluster.connect('yelp_weather_data',wait_for_all_pools=True) #json - name of keyspace
    session.execute('USE yelp_weather_data')

    query = "INSERT INTO user_recommendation(user_id , max_corr_category, max_corr_user ) VALUES (?,?,?)"
    prepared = session.prepare(query)

    for index, item in df.iterrows():
        print(index)
        print( str(item['user_id'])+ str(item['max_corr_category']) + str(item['max_corr_user']))
        session.execute(prepared, ( str(item['user_id']), str(item['max_corr_category']), str(item['max_corr_user'])))
    

In [ ]:
extractedDf = pd.read_csv(outputFilePath+'user_category_user.csv')
print(extractedDf.head())

In [2]:
print(df.columns)

Index(['user_id', 'Food', 'Shopping', 'Home Services', 'Beauty & Spas',
       'Health & Medical', 'Bars', 'Local Services', 'Nightlife', 'Automotive',
       'Event Planning & Services', 'Active Life', 'Fashion', 'Coffee & Tea',
       'Professional Services', 'Arts & Entertainment', 'Hotels & Travel',
       'Real Estate', 'Pet Services', 'Grocery', 'Education', 'Contractors',
       'Financial Services', 'Flowers & Gifts', 'Clothing & accessories',
       'Electronics', 'Community Service/Non-Profit',
       'Public Services & Government', 'Laundry Services', 'Transportation',
       'Lawyers', 'Electricians', 'Religious Organizations', 'Miscelleanous'],
      dtype='object')


In [ ]:
#df.rename(columns={"A": "a", "B": "c"})

from cassandra.cluster import Cluster
import pandas as pd

if __name__ == "__main__":
    cluster = Cluster(['127.0.0.1'], port = 9042)
    session = cluster.connect('yelp_weather_data',wait_for_all_pools=True) #json - name of keyspace
    session.execute('USE yelp_weather_data')

    query = "INSERT INTO user_category(user_id,food,shopping,home_services,beauty_and_spas,health_and_medical,bars,local_services,nightlife,automotive,professional_services,arts_and_entertainment,hotels_and_travel,financial_services,flowers_and_gifts,clothing_and_accessories,electronics,community_service_non_profit,public_services_and_government,laundry_services,transportation,lawyers,electricians,religious_organizations,miscellaneous)" 
    values = " VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    query = query + values
    prepared = session.prepare(query)

    for index, item in df.iterrows():
        print(index)
       # print( str(item['user_id'])+ str(item['max_corr_category']) + str(item['max_corr_user']))
        session.execute(prepared, ( str(item['user_id']), int(item['Food']),int(item['Shopping']),int(item['Home Services']),int(item['Beauty & Spas']),int(item['Health & Medical']),int(item['Bars']),int(item['Local Services']),int(item['Nightlife']),int(item['Automotive']),int(item['Professional Services']),	int(item['Arts & Entertainment']),int(item['Hotels & Travel']),	int(item['Financial Services']),	int(item['Flowers & Gifts']),	int(item['Clothing & accessories']),	int(item['Electronics']),	int(item['Community Service/Non-Profit']),	int(item['Public Services & Government']),	int(item['Laundry Services']),	int(item['Transportation']),	int(item['Lawyers']),	int(item['Electricians']),	int(item['Religious Organizations']),	int(item['Miscelleanous'])
                                  ))
    

In [13]:
#retrieve data from Cassandra DB:
from cassandra.cluster import Cluster
import pandas as pd

if __name__ == "__main__":
    cluster = Cluster(['127.0.0.1'], port = 9042)
    keyspace = 'yelp_weather_data'
    session = cluster.connect(keyspace, wait_for_all_pools=True) #json - name of keyspace
    query = "SELECT * from user_category"
    df = pd.DataFrame(list(session.execute(query)))

    print(df.head())
    

                  user_id  arts_and_entertainment  automotive  bars  \
0  TTRVdTXKcq-xn6-1IWbwEw                       0           0     5   
1  th7tfMsh74S6sa_BWOt8YA                       3           5     3   
2  yKYpt0bcSVD5W4FoPCY5iQ                       0           0     0   
3  zH6racrZ7rUqqv_Qy6dZDQ                       3           3     3   
4  jAVtSgE5L-Dt6_I5FIiVGA                       0           5     4   

   beauty_and_spas  clothing_and_accessories  community_service_non_profit  \
0                0                         0                             0   
1                4                         5                             4   
2                5                         0                             0   
3                4                         2                             4   
4                0                         0                             0   

   electricians  electronics  financial_services  ...  laundry_services  \
0             0            0 